This tutorial and the assets can be downloaded as part of the [Wallaroo Tutorials repository](https://github.com/WallarooLabs/Wallaroo_Tutorials/tree/main/wallaroo-model-cookbooks/aloha).

## Aloha Demo

In this notebook we will walk through a simple pipeline deployment to inference on a model. For this example we will be using an open source model that uses an [Aloha CNN LSTM model](https://www.researchgate.net/publication/348920204_Using_Auxiliary_Inputs_in_Deep_Learning_Models_for_Detecting_DGA-based_Domain_Names) for classifying Domain names as being either legitimate or being used for nefarious purposes such as malware distribution.  

For our example, we will perform the following:

* Create a workspace for our work.
* Upload the Aloha model.
* Create a pipeline that can ingest our submitted data, submit it to the model, and export the results
* Run a sample inference through our pipeline by loading a file
* Run a sample inference through our pipeline's URL and store the results in a file.

All sample data and models are available through the [Wallaroo Quick Start Guide Samples repository](https://github.com/WallarooLabs/quickstartguide_samples).

## Open a Connection to Wallaroo

The first step is to connect to Wallaroo through the Wallaroo client.  The Python library is included in the Wallaroo install and available through the Jupyter Hub interface provided with your Wallaroo environment.

This is accomplished using the `wallaroo.Client()` command, which provides a URL to grant the SDK permission to your specific Wallaroo environment.  When displayed, enter the URL into a browser and confirm permissions.  Store the connection into a variable that can be referenced later.

In [35]:
import wallaroo
from wallaroo.object import EntityNotFoundError

# used to display dataframe information without truncating
import pandas as pd
pd.set_option('display.max_colwidth', None)
import pyarrow as pa

# to display dataframe tables
from IPython.display import display

In [2]:
# Login through local Wallaroo instance

# wl = wallaroo.Client()

# SSO login through keycloak

wallarooPrefix = "sparkly-apple-3026"
wallarooSuffix = "wallaroo.community"

wl = wallaroo.Client(api_endpoint=f"https://{wallarooPrefix}.api.{wallarooSuffix}", 
                    auth_endpoint=f"https://{wallarooPrefix}.keycloak.{wallarooSuffix}", 
                    auth_type="sso")

### Arrow Support

As of the 2023.1 release, Wallaroo provides support for dataframe and Arrow for inference inputs.  This tutorial allows users to adjust their experience based on whether they have enabled Arrow support in their Wallaroo instance or not.

If Arrow support has been enabled, `arrowEnabled=True`. If disabled or you're not sure, set it to `arrowEnabled=False`

The examples below will be shown in an arrow enabled environment.

In [3]:
import os
# Only set the below to make the OS environment ARROW_ENABLED to TRUE.  Otherwise, leave as is.
os.environ["ARROW_ENABLED"]="True"

if "ARROW_ENABLED" not in os.environ or os.environ["ARROW_ENABLED"].casefold() == "False".casefold():
    arrowEnabled = False
else:
    arrowEnabled = True
print(arrowEnabled)

True


## Create the Workspace

We will create a workspace to work in and call it the "alohaworkspace", then set it as current workspace environment.  We'll also create our pipeline in advance as `alohapipeline`.  The model name and the model file will be specified for use in later steps.

To allow this tutorial to be run multiple times or by multiple users in the same Wallaroo instance, a random 4 character prefix will be added to the workspace, pipeline, and model.

In [4]:
import string
import random

# make a random 4 character prefix
prefix= ''.join(random.choice(string.ascii_lowercase) for i in range(4))
workspace_name = f'{prefix}alohaworkspace'
pipeline_name = f'{prefix}alohapipeline'
model_name = f'{prefix}alohamodel'
model_file_name = './alohacnnlstm.zip'

In [5]:
def get_workspace(name):
    workspace = None
    for ws in wl.list_workspaces():
        if ws.name() == name:
            workspace= ws
    if(workspace == None):
        workspace = wl.create_workspace(name)
    return workspace

def get_pipeline(name):
    try:
        pipeline = wl.pipelines_by_name(pipeline_name)[0]
    except EntityNotFoundError:
        pipeline = wl.build_pipeline(pipeline_name)
    return pipeline

In [6]:
wl.list_workspaces()

Name,Created At,Users,Models,Pipelines
john.hansarick@wallaroo.ai - Default Workspace,2023-02-17 20:36:12,['john.hansarick@wallaroo.ai'],5,2
testautoconversion,2023-02-21 17:02:22,['john.hansarick@wallaroo.ai'],2,0
kerasautoconvertworkspace,2023-02-21 18:09:28,['john.hansarick@wallaroo.ai'],1,1
externalkerasautoconvertworkspace,2023-02-21 18:16:14,['john.hansarick@wallaroo.ai'],1,1
ccfraudcomparisondemo,2023-02-21 18:31:10,['john.hansarick@wallaroo.ai'],6,3
isolettest,2023-02-21 21:24:33,['john.hansarick@wallaroo.ai'],1,1
bikedayevalworkspace,2023-02-22 16:42:58,['john.hansarick@wallaroo.ai'],1,1
xgboost-classification-autoconvert-workspace,2023-02-22 17:28:52,['john.hansarick@wallaroo.ai'],1,1
xgboost-regression-autoconvert-workspace,2023-02-22 17:36:30,['john.hansarick@wallaroo.ai'],1,1
housepricing,2023-02-22 18:28:40,['john.hansarick@wallaroo.ai'],3,1


In [7]:
workspace = get_workspace(workspace_name)

wl.set_current_workspace(workspace)

aloha_pipeline = get_pipeline(pipeline_name)
aloha_pipeline

name,yarbalohapipeline
created,2023-03-20 19:33:46.153485+00:00
last_updated,2023-03-20 19:33:46.153485+00:00
deployed,(none)
tags,
versions,050a5b70-a4db-45b3-aebd-4d4d0efd25de
steps,


We can verify the workspace is created the current default workspace with the `get_current_workspace()` command.

In [8]:
wl.get_current_workspace()

{'name': 'yarbalohaworkspace', 'id': 161, 'archived': False, 'created_by': '138bd7e6-4dc8-4dc1-a760-c9e721ef3c37', 'created_at': '2023-03-20T19:33:45.54861+00:00', 'models': [], 'pipelines': [{'name': 'yarbalohapipeline', 'create_time': datetime.datetime(2023, 3, 20, 19, 33, 46, 153485, tzinfo=tzutc()), 'definition': '[]'}]}

# Upload the Models

Now we will upload our models.  Note that for this example we are applying the model from a .ZIP file.  The Aloha model is a [protobuf](https://developers.google.com/protocol-buffers) file that has been defined for evaluating web pages, and we will configure it to use data in the `tensorflow` format.

In [9]:
model = wl.upload_model(model_name, model_file_name).configure("tensorflow")

## Deploy a model

Now that we have a model that we want to use we will create a deployment for it. 

We will tell the deployment we are using a tensorflow model and give the deployment name and the configuration we want for the deployment.

To do this, we'll create our pipeline that can ingest the data, pass the data to our Aloha model, and give us a final output.  We'll call our pipeline `aloha-test-demo`, then deploy it so it's ready to receive data.  The deployment process usually takes about 45 seconds.

* **Note**:  If you receive an error that the pipeline could not be deployed because there are not enough resources, undeploy any other pipelines and deploy this one again.  This command can quickly undeploy all pipelines to regain resources.  We recommend **not** running this command in a production environment since it will cancel any running pipelines:

```python
for p in wl.list_pipelines(): p.undeploy()
```

In [10]:
aloha_pipeline.add_model_step(model)

name,yarbalohapipeline
created,2023-03-20 19:33:46.153485+00:00
last_updated,2023-03-20 19:33:46.153485+00:00
deployed,(none)
tags,
versions,050a5b70-a4db-45b3-aebd-4d4d0efd25de
steps,


In [11]:
aloha_pipeline.deploy()

WaitForDeployError: Deployment failed. See status for details.
Status: {'status': 'Error', 'details': [], 'engines': [{'ip': None, 'name': 'engine-64774fc4f6-mhw2k', 'status': 'Pending', 'reason': '0/4 nodes are available: 1 Insufficient cpu, 4 Insufficient memory. preemption: 0/4 nodes are available: 4 No preemption victims found for incoming pod.', 'details': ['0/4 nodes are available: 1 Insufficient cpu, 4 Insufficient memory. preemption: 0/4 nodes are available: 4 No preemption victims found for incoming pod.'], 'pipeline_statuses': None, 'model_statuses': None}], 'engine_lbs': [{'ip': '10.48.1.66', 'name': 'engine-lb-86bc6bd77b-dc9xp', 'status': 'Running', 'reason': None, 'details': []}], 'sidekicks': []}

We can verify that the pipeline is running and list what models are associated with it.

In [ ]:
aloha_pipeline.status()

{'status': 'Running',
 'details': [],
 'engines': [{'ip': '10.244.0.41',
   'name': 'engine-74fb5d899d-q885n',
   'status': 'Running',
   'reason': None,
   'details': [],
   'pipeline_statuses': {'pipelines': [{'id': 'uxelalohapipeline',
      'status': 'Running'}]},
   'model_statuses': {'models': [{'name': 'uxelalohamodel',
      'version': '3f354fb2-4220-4873-9418-6debc06ada57',
      'sha': 'd71d9ffc61aaac58c2b1ed70a2db13d1416fb9d3f5b891e5e4e2e97180fe22f8',
      'status': 'Running'}]}}],
 'engine_lbs': [{'ip': '10.244.2.16',
   'name': 'engine-lb-ddd995646-j5nr2',
   'status': 'Running',
   'reason': None,
   'details': []}],
 'sidekicks': []}

## Interferences

### Infer 1 row

Now that the pipeline is deployed and our Aloha model is in place, we'll perform a smoke test to verify the pipeline is up and running properly.  We'll use the `infer_from_file` command to load a single encoded URL into the inference engine and print the results back out.

The result should tell us that the tokenized URL is legitimate (0) or fraud (1).  This sample data should return close to 1 in `out.main`.

In [18]:
high_fraud = smoke_test = pd.DataFrame.from_records(
    [
    {
        "text_input":[
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            0,
            28,
            16,
            32,
            23,
            29,
            32,
            30,
            19,
            26,
            17
        ]
    }
]
)

result = aloha_pipeline.infer(high_fraud)
display(result.loc[:, ["time","out.main"]])

,time,out.main
0,2023-03-20 19:41:10.323,[0.997564]


In [33]:
result = aloha_pipeline.infer_from_file('./data/data_1k.arrow')
display(result.loc[:, ["time","out.main"]])

InferenceError: Inference failed: 
	InternalError: There was an error sending data to the pipeline: Received none result for Model Inference

In [36]:
arrow_file_name = "./data/data_1k.arrow"
with pa.ipc.open_file(arrow_file_name) as source:
    table = source.read_all() # to get pyarrow table
    # table_df = source.read_pandas() # to get pandas dataframe
    display(table)
    # display(table_df)

pyarrow.Table
text_input: fixed_size_list<item: int64>[50]
  child 0, item: int64
----
text_input: [[[0,0,0,0,0,...,32,30,19,26,17],[0,0,0,0,0,...,29,12,36,31,12],...,[0,0,0,0,0,...,35,16,35,27,16],[0,0,0,0,0,...,24,29,14,36,13]]]

### Batch Inference

Now that our smoke test is successful, let's really give it some data.  We have two inference files we can use:

* `data_1k.arrow`:  Contains 10,000 inferences
* `data_25k.arrow`: Contains 25,000 inferences

When Apache Arrow tables are submitted to a Wallaroo Pipeline, the inference is processed natively as an Arrow table, and the results are returned as an Arrow table.  This allows for faster data processing than with JSON files or DataFrame objects.

We'll pipe the `data_25k.arrow` file through the `aloha_pipeline` deployment URL, and place the results in a file named `response.arrow`.  We'll also display the time this takes.  Note that for larger batches of 50,000 inferences or more can be difficult to view in Jupyter Hub because of its size, so we'll only display the first five rows.

* **IMPORTANT NOTE**:  The `_deployment._url()` method will return an **internal** URL when using Python commands from within the Wallaroo instance - for example, the Wallaroo JupyterHub service.  When connecting via an external connection, `_deployment._url()` returns an **external** URL.  External URL connections requires [the authentication be included in the HTTP request](https://docs.wallaroo.ai/wallaroo-developer-guides/wallaroo-api-guide/), and that [Model Endpoints Guide](https://docs.wallaroo.ai/wallaroo-operations-guide/wallaroo-configuration/wallaroo-model-endpoints-guide/) external endpoints are enabled in the Wallaroo configuration options.

In [25]:
inference_url = aloha_pipeline._deployment._url()
inference_url

'https://sparkly-apple-3026.api.wallaroo.community/v1/api/pipelines/infer/yarbalohapipeline-416'

In [26]:
connection =wl.mlops().__dict__
token = connection['token']
token

'eyJhbGciOiJSUzI1NiIsInR5cCIgOiAiSldUIiwia2lkIiA6ICJjTGZaYmhVQWl0a210Z0VLV0l1NnczTWlXYmUzWjc3cHdqVjJ2QWM2WUdZIn0.eyJleHAiOjE2NzkzNDE0MDQsImlhdCI6MTY3OTM0MTM0NCwiYXV0aF90aW1lIjoxNjc5MzM4MDUzLCJqdGkiOiJiOTA4Zjk0MS1hMGM5LTQ1ZjUtOWNmYy02YTQ4NmJhNTUxOTEiLCJpc3MiOiJodHRwczovL3NwYXJrbHktYXBwbGUtMzAyNi5rZXljbG9hay53YWxsYXJvby5jb21tdW5pdHkvYXV0aC9yZWFsbXMvbWFzdGVyIiwiYXVkIjpbIm1hc3Rlci1yZWFsbSIsImFjY291bnQiXSwic3ViIjoiMTM4YmQ3ZTYtNGRjOC00ZGMxLWE3NjAtYzllNzIxZWYzYzM3IiwidHlwIjoiQmVhcmVyIiwiYXpwIjoic2RrLWNsaWVudCIsInNlc3Npb25fc3RhdGUiOiI5OGFjYTBhNi03YmRhLTQ3MjAtOGQzNi1kNWZmNjA1NTY4MWUiLCJhY3IiOiIwIiwicmVhbG1fYWNjZXNzIjp7InJvbGVzIjpbImNyZWF0ZS1yZWFsbSIsImRlZmF1bHQtcm9sZXMtbWFzdGVyIiwib2ZmbGluZV9hY2Nlc3MiLCJhZG1pbiIsInVtYV9hdXRob3JpemF0aW9uIl19LCJyZXNvdXJjZV9hY2Nlc3MiOnsibWFzdGVyLXJlYWxtIjp7InJvbGVzIjpbInZpZXctcmVhbG0iLCJ2aWV3LWlkZW50aXR5LXByb3ZpZGVycyIsIm1hbmFnZS1pZGVudGl0eS1wcm92aWRlcnMiLCJpbXBlcnNvbmF0aW9uIiwiY3JlYXRlLWNsaWVudCIsIm1hbmFnZS11c2VycyIsInF1ZXJ5LXJlYWxtcyIsInZpZXctYXV0aG9yaXphdGlvbiI

In [27]:
dataFile="./data/data_25k.arrow"
contentType="application/vnd.apache.arrow.file"

In [28]:
!curl -X POST {inference_url} -H "Authorization: Bearer {token}" -H "Content-Type:{contentType}" --data-binary @{dataFile} > curl_response.df

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 9748k  100   132  100 9748k     77  5725k  0:00:01  0:00:01 --:--:-- 5734k


In [29]:
cc_data_from_file =  pd.read_json('./curl_response.txt', orient="records")
display(cc_data_from_file.head(5))

,error,error_type
0,"""Error deserializing json"": ""arrow i/o error: InvalidFooter""",BadRequest


## Undeploy Pipeline

When finished with our tests, we will undeploy the pipeline so we have the Kubernetes resources back for other tasks.  Note that if the deployment variable is unchanged aloha_pipeline.deploy() will restart the inference engine in the same configuration as before.

In [37]:
aloha_pipeline.undeploy()